# Simulación de un sistema de colas con servidores paralelos
# (Ejemplo 7.4 del libro "Simulation, Fifth Edition by Sheldon M. Ross")

Este archivo implementa una simulación de eventos discretos para un sistema de colas con las siguientes características:

- Los clientes llegan al sistema de acuerdo con una distribución de tiempos entre llegadas.
- El sistema cuenta con varios servidores paralelos.
- Cuando un cliente llega:
  - Si todos los servidores están ocupados, el cliente entra en una cola FIFO (primero en llegar, primero en ser atendido).
  - Si algún servidor está libre, el cliente entra directamente a servicio con el primer servidor que esté libre según el orden de servidores.
- Cuando un servidor termina de atender a un cliente:
  - El cliente abandona el sistema.
  - Si hay clientes en cola, el siguiente en orden entra al servidor que se ha liberado.
- Cada servidor tiene su propia distribución de tiempos de servicio.
- Se realiza un seguimiento del:
  - Tiempo total en el sistema de cada cliente (desde que llega hasta que se va).
  - Número de servicios completados por cada servidor.

La simulación está basada en el paradigma de simulación de eventos discretos, manteniendo una lista de eventos con los siguientes tipos:
- `arrival`: llegada de un nuevo cliente.
- `departure`: finalización de servicio en el servidor especificado en el evento.

Esta simulación permite estudiar el comportamiento del sistema bajo distintas distribuciones de llegada y servicio, y analizar métricas como tiempo promedio en el sistema, carga de los servidores, y longitud promedio de la cola.


In [ ]:
# Importar librerías a utilizar
import random
import heapq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- Evento ---
class Event:
    def __init__(self, time, e_type, server_id=None):
        self.time = time
        self.type = e_type  # 'arrival', 'departure'
        self.server_id = server_id

    def __lt__(self, other):
        return self.time < other.time  # Sobrecarga de operador '<' para indicar a python como comparar dos eventos usando el tiempo

# --- Simulador ---
def simulate_queueing_system(servers_ammount, arrival_rate, service_time_rates, time_limit):
    actual_time = 0                             # tiempo actual
    event_list = []                             # lista de eventos (heapq)
    queue = []                                  # cola de espera
    servers = [None] * servers_ammount          # cada servidor es inicializado vacío
    standby_times = []                          # tiempo que cada cliente esperó antes de ser procesada
    system_times = []                           # tiempos totales en el sistema
    processing_time = [0] * servers_ammount     # tiempo acumulado de procesamiento para cada servidor
    last_time = 0                               # último tiempo registrado en el sistema

    # insertando la primera llegada al sistema de una solicitud
    heapq.heappush(event_list, Event(random.expovariate(arrival_rate), 'arrival'))

    # empezando a procesar los eventos del sistema
    while event_list:
        event = heapq.heappop(event_list)   # procesando el evento que corresponde
        actual_time = event.time            # actualizando tiempo del evento
        # si no pasamos del límite de tiempo, detén el bucle
        if actual_time > time_limit:
            break
        # actualizando tiempo de procesamiento de servidores ocupados
        for i in range(servers_ammount):
            if servers[i] is not None:
                processing_time[i] = processing_time[i] + (actual_time-last_time)
            last_time = actual_time # actualizando tiempo anterior

        # verificando caso del evento de llegada
        if event.type == 'arrival':
            # detectando si hay servidores libres
            free_server = None
            for i in range(servers_ammount):
                if servers[i] is None:
                    free_server = i
                    break
            # si existe algún servidor libre, agrega la solicitud a ese servidor
            if free_server is not None:
                service_time = random.expovariate(service_time_rates[free_server])    # genera tiempo de procesamiento aleatorio
                departure = actual_time + service_time                  # calcula el tiempo de salida
                servers[free_server] = departure                        # agrega el tiempo de salida calculado al servidor
                heapq.heappush(event_list, Event(departure, 'departure', free_server))  # agrega el evento de salida a la lista de eventos
                standby_times.append(0)                                 # agrega 0 como tiempo de espera de la solicitud
                system_times.append(service_time)                       # solo servicio, sin espera
            # si no existe ningún servidor libre, agrega la solicitud del cliente a la cola
            else:
                queue.append(actual_time) # guardar tiempo de llegada en la cola
            next_arrival = actual_time + random.expovariate(arrival_rate)
            heapq.heappush(event_list, Event(next_arrival, 'arrival'))

        # verificando caso del evento de salida
        elif event.type == 'departure':
            # si el evento es una salida, finaliza el proceso en el procesador
            servers[event.server_id] = None
            # si hay solicitudes en cola, que entre la siguiente al procesador liberado
            if queue:
                client_arrival = queue.pop(0)
                waiting_time = actual_time - client_arrival
                standby_times.append(waiting_time)
                service_time = random.expovariate(service_time_rates[event.server_id])
                departure = actual_time + service_time
                servers[event.server_id] = departure
                heapq.heappush(event_list, Event(departure, 'departure', event.server_id))
                system_times.append(waiting_time + service_time)  # total en sistema

    # Calculando tiempo de espera y tiempo total en el sistema para cada cliente
    if standby_times:
        average_standby_time = sum(standby_times) / len(standby_times) 
    else:
        average_standby_time = 0

    if average_system_times:
        average_system_times = sum(system_times) / len(system_times)
    else:
        average_system_times = 0

    # Retornar
    return average_standby_time, average_system_times, len(standby_times), processing_time